In [60]:
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import pandas as pd 
import random
import math
import time
import os
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
import operator 
plt.style.use('fivethirtyeight')
%matplotlib inline

In [61]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recoveries_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [62]:
# Analyse data

In [63]:
## Test same columns
print(np.all(confirmed_df.columns == deaths_df.columns))
print(np.all(deaths_df.columns == recoveries_df.columns))

True
True


In [64]:
print(confirmed_df['Country/Region'].unique().size, 
deaths_df['Country/Region'].unique().size,
recoveries_df['Country/Region'].unique().size)

188 188 188


In [65]:
for i in range(recoveries_df['Country/Region'].unique().size):
    print(
        recoveries_df['Country/Region'].unique()[i] == deaths_df['Country/Region'].unique()[i],
        confirmed_df['Country/Region'].unique()[i], 
        deaths_df['Country/Region'].unique()[i],
        recoveries_df['Country/Region'].unique()[i])
# décallage dans les indices pour deaths_df
# nom non consistant (ex : republic)
# -> simplification, selections de quelque pays


True Afghanistan Afghanistan Afghanistan
True Albania Albania Albania
True Algeria Algeria Algeria
True Andorra Andorra Andorra
True Angola Angola Angola
True Antigua and Barbuda Antigua and Barbuda Antigua and Barbuda
True Argentina Argentina Argentina
True Armenia Armenia Armenia
True Australia Australia Australia
True Austria Austria Austria
True Azerbaijan Azerbaijan Azerbaijan
True Bahamas Bahamas Bahamas
True Bahrain Bahrain Bahrain
True Bangladesh Bangladesh Bangladesh
True Barbados Barbados Barbados
True Belarus Belarus Belarus
True Belgium Belgium Belgium
False Benin Benin Belize
False Bhutan Bhutan Benin
False Bolivia Bolivia Bhutan
False Bosnia and Herzegovina Bosnia and Herzegovina Bolivia
False Brazil Brazil Bosnia and Herzegovina
False Brunei Brunei Brazil
False Bulgaria Bulgaria Brunei
False Burkina Faso Burkina Faso Bulgaria
False Cabo Verde Cabo Verde Burkina Faso
False Cambodia Cambodia Cabo Verde
False Cameroon Cameroon Cambodia
False Canada Canada Cameroon
False Cen

In [66]:
cols = confirmed_df.keys()
print(cols)

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '7/19/20', '7/20/20', '7/21/20', '7/22/20', '7/23/20', '7/24/20',
       '7/25/20', '7/26/20', '7/27/20', '7/28/20'],
      dtype='object', length=193)


In [67]:
confirmed = confirmed_df.loc[:, cols[4]:cols[-1]]
deaths = deaths_df.loc[:, cols[4]:cols[-1]]
recoveries = recoveries_df.loc[:, cols[4]:cols[-1]]


In [68]:
dates = confirmed.keys()
world_cases = []
total_deaths = [] 
mortality_rate = []
recovery_rate = [] 
total_recovered = [] 
total_active = [] 

china_cases = [] 
italy_cases = []
us_cases = [] 
spain_cases = [] 
france_cases = [] 
germany_cases = [] 
uk_cases = [] 
russia_cases = [] 
brazil_cases = []
india_cases = []
peru_cases = [] 

china_deaths = [] 
italy_deaths = []
us_deaths = [] 
spain_deaths = [] 
france_deaths = [] 
germany_deaths = [] 
uk_deaths = [] 
russia_deaths = []
brazil_deaths = [] 
india_deaths = []
peru_deaths = []

china_recoveries = [] 
italy_recoveries = []
us_recoveries = [] 
spain_recoveries = [] 
france_recoveries = [] 
germany_recoveries = [] 
uk_recoveries = [] 
russia_recoveries = [] 
brazil_recoveries = [] 
india_recoveries = [] 
peru_recoveries = [] 

for i in dates:
    confirmed_sum = confirmed[i].sum()
    death_sum = deaths[i].sum()
    recovered_sum = recoveries[i].sum()
    
    # confirmed, deaths, recovered, and active
    world_cases.append(confirmed_sum)
    total_deaths.append(death_sum)
    total_recovered.append(recovered_sum)
    total_active.append(confirmed_sum-death_sum-recovered_sum)
    
    # calculate rates
    mortality_rate.append(death_sum/confirmed_sum)
    recovery_rate.append(recovered_sum/confirmed_sum)

    # case studies 
    china_cases.append(confirmed_df[confirmed_df['Country/Region']=='China'][i].sum())
    italy_cases.append(confirmed_df[confirmed_df['Country/Region']=='Italy'][i].sum())
    us_cases.append(confirmed_df[confirmed_df['Country/Region']=='US'][i].sum())
    spain_cases.append(confirmed_df[confirmed_df['Country/Region']=='Spain'][i].sum())
    france_cases.append(confirmed_df[confirmed_df['Country/Region']=='France'][i].sum())
    germany_cases.append(confirmed_df[confirmed_df['Country/Region']=='Germany'][i].sum())
    uk_cases.append(confirmed_df[confirmed_df['Country/Region']=='United Kingdom'][i].sum())
    russia_cases.append(confirmed_df[confirmed_df['Country/Region']=='Russia'][i].sum())
    brazil_cases.append(confirmed_df[confirmed_df['Country/Region']=='Brazil'][i].sum())
    india_cases.append(confirmed_df[confirmed_df['Country/Region']=='India'][i].sum())
    peru_cases.append(confirmed_df[confirmed_df['Country/Region']=='Peru'][i].sum())
    
    # moving average for case studies 
    china_deaths.append(deaths_df[deaths_df['Country/Region']=='China'][i].sum())
    italy_deaths.append(deaths_df[deaths_df['Country/Region']=='Italy'][i].sum())
    us_deaths.append(deaths_df[deaths_df['Country/Region']=='US'][i].sum())
    spain_deaths.append(deaths_df[deaths_df['Country/Region']=='Spain'][i].sum())
    france_deaths.append(deaths_df[deaths_df['Country/Region']=='France'][i].sum())
    germany_deaths.append(deaths_df[deaths_df['Country/Region']=='Germany'][i].sum())
    uk_deaths.append(deaths_df[deaths_df['Country/Region']=='United Kingdom'][i].sum())
    russia_deaths.append(deaths_df[deaths_df['Country/Region']=='Russia'][i].sum())
    brazil_deaths.append(deaths_df[deaths_df['Country/Region']=='Brazil'][i].sum())
    india_deaths.append(deaths_df[deaths_df['Country/Region']=='India'][i].sum())
    peru_deaths.append(deaths_df[deaths_df['Country/Region']=='Peru'][i].sum())
    
    china_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='China'][i].sum())
    italy_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='Italy'][i].sum())
    us_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='US'][i].sum())
    spain_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='Spain'][i].sum())
    france_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='France'][i].sum())
    germany_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='Germany'][i].sum())
    uk_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='United Kingdom'][i].sum())
    russia_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='Russia'][i].sum())
    brazil_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='Brazil'][i].sum())
    india_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='India'][i].sum())
    peru_recoveries.append(recoveries_df[recoveries_df['Country/Region']=='Peru'][i].sum())

In [69]:
recoveries = [["china"]+china_recoveries, 
["italy"]+italy_recoveries,
["us"]+us_recoveries, 
["spain"]+spain_recoveries, 
["france"]+france_recoveries, 
["germany"]+germany_recoveries, 
["uk"]+uk_recoveries, 
["russia"]+russia_recoveries, 
["brazil"]+brazil_recoveries, 
["india"]+india_recoveries, 
["peru"]+peru_recoveries]


In [70]:
deaths = [["china"]+china_deaths, 
["italy"]+italy_deaths,
["us"]+us_deaths, 
["spain"]+spain_deaths, 
["france"]+france_deaths, 
["germany"]+germany_deaths, 
["uk"]+uk_deaths, 
["russia"]+russia_deaths, 
["brazil"]+brazil_deaths, 
["india"]+india_deaths, 
["peru"]+peru_deaths]


In [71]:
cases = [["china"]+china_cases, 
["italy"]+italy_cases,
["us"]+us_cases, 
["spain"]+spain_cases, 
["france"]+france_cases, 
["germany"]+germany_cases, 
["uk"]+uk_cases, 
["russia"]+russia_cases, 
["brazil"]+brazil_cases, 
["india"]+india_cases, 
["peru"]+peru_cases]


In [72]:
columns = np.append(["country"],dates)
recoveries_df = pd.DataFrame(data=recoveries, columns=columns)
deaths_df = pd.DataFrame(data=deaths, columns=columns)
cases_df = pd.DataFrame(data=cases, columns=columns)

In [73]:
index=["cases", "total_deaths", "total_recovery", "total_active", "mortality_rate", "recovery_rate"]
data = [
    world_cases,
    total_deaths,
    total_recovered,
    total_active,
    mortality_rate,
    recovery_rate
]
world_summary = pd.DataFrame(data=data, index=index, columns=dates)

In [74]:
recoveries_df.to_csv(os.path.join("data",'recoveries_data.csv'))
deaths_df.to_csv(os.path.join("data",'deaths_data.csv'))
cases_df.to_csv(os.path.join("data",'cases_data.csv'))
world_summary.to_csv(os.path.join("data",'world_data.csv'))

In [81]:
confirmed_cases = cases_df.loc[cases_df["country"] == "china"]["7/27/20"][0]
print(confirmed_cases)
#cases_df.loc[cases_df["country"] == str(country)][str(date)][0]

86783
